In [5]:
import numpy as np
from psopy import _minimize_pso
from psopy import init_feasible, gen_confunc

fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2
fun_ = lambda x: np.apply_along_axis(fun, 1, x)

cons = ({'type': 'ineq', 'fun': lambda x:  x[0] - 2 * x[1] + 2},
        {'type': 'ineq', 'fun': lambda x: -x[0] - 2 * x[1] + 6},
        {'type': 'ineq', 'fun': lambda x: -x[0] + 2 * x[1] + 2},
        {'type': 'ineq', 'fun': lambda x: x[0]},
        {'type': 'ineq', 'fun': lambda x: x[1]})

x0 = init_feasible(cons, low=0., high=2., shape=(1000, 2))
confunc = gen_confunc(cons)

options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4., 'stable_iter': 50}
res = _minimize_pso(fun_, x0, confunc=confunc, **options)

print(res.x)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [13]:
fun_(x0)[0]

5.1025795193691215

In [11]:
x0[0]

array([1.77711854, 0.37899404])

In [12]:
(1.77711854 - 1)**2 + (0.37899404 - 2.5)**2

5.1025795075672535

In [14]:
import numpy as np
from pyswarm import pso

# Define the objective (to be minimize)
def weight(x, *args):
    H, d, t = x
    B, rho, E, P = args
    return rho*2*np.pi*d*t*np.sqrt((B/2)**2 + H**2)

# Setup the constraint functions
def yield_stress(x, *args):
    H, d, t = x
    B, rho, E, P = args
    return (P*np.sqrt((B/2)**2 + H**2))/(2*t*np.pi*d*H)

def buckling_stress(x, *args):
    H, d, t = x
    B, rho, E, P = args
    return (np.pi**2*E*(d**2 + t**2))/(8*((B/2)**2 + H**2))

def deflection(x, *args):
    H, d, t = x
    B, rho, E, P = args
    return (P*np.sqrt((B/2)**2 + H**2)**3)/(2*t*np.pi*d*H**2*E)

def constraints(x, *args):
    strs = yield_stress(x, *args)
    buck = buckling_stress(x, *args)
    defl = deflection(x, *args)
    return [100 - strs, buck - strs, 0.25 - defl]

# Define the other parameters
B = 60  # inches
rho = 0.3  # lb/in^3
E = 30000  # kpsi (1000-psi)
P = 66  # kip (1000-lbs, force)
args = (B, rho, E, P)

# Define the lower and upper bounds for H, d, t, respectively
lb = [10, 1, 0.01]
ub = [30, 3, 0.25]

xopt, fopt = pso(weight, lb, ub, f_ieqcons=constraints, args=args)

# The optimal input values are approximately
#     xopt = [29, 2.4, 0.06]
# with function values approximately
#     weight          = 12 lbs
#     yield stress    = 100 kpsi (binding constraint)
#     buckling stress = 150 kpsi
#     deflection      = 0.2 in

Stopping search: maximum iterations reached --> 100


In [15]:
xopt

array([26.76558869,  2.22469216,  0.07092477])